# Clean Slate: Estimating offenses eligible for expungement under varying conditions
> Prepared by [Laura Feeney](https://github.com/laurafeeney) for Code for Boston's [Clean Slate project](https://github.com/codeforboston/clean-slate).

## Purpose
Exploring duplicate charges, dispositions, and moves from one court to another. 

-----

### Step 0
Import data, programs, etc.

-----

In [1]:
import pandas as pd
pd.set_option("display.max_rows", 200)
import numpy as np
import regex as re
import glob, os
import datetime 
from datetime import date 

In [2]:
# processed individual-level data from NW district with expungability.

suff = pd.read_csv('../../data/processed/merged_suff.csv', encoding='cp1252',
                    dtype={'Analysis notes':str, 'extra_criteria':str, 'Expungeable': str}) 
suff.info()

nw = pd.read_csv('../../data/processed/merged_nw.csv', encoding='cp1252',
                    dtype={'Analysis notes':str, 'extra_criteria':str, 'Expungeable': str}) 
nw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303270 entries, 0 to 303269
Data columns (total 21 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   Person ID                       303270 non-null  object
 1   Court                           303270 non-null  object
 2   Offense Date                    300877 non-null  object
 3   Filed                           303270 non-null  object
 4   Status                          303270 non-null  object
 5   Count                           303270 non-null  int64 
 6   Charge                          303270 non-null  object
 7   Type Crime                      303270 non-null  object
 8   Code Ucc Ctgry                  262955 non-null  object
 9   Disposition                     251710 non-null  object
 10  Description Disposition Reason  212684 non-null  object
 11  Dispo Date                      251300 non-null  object
 12  CMRoffense                    

In [3]:
print("Suffolk")
#drop CMR offenses, remaining unique people
suff = suff.loc[suff['CMRoffense']=='no'].copy()
print(suff['Person ID'].nunique())
total_people_suff = suff['Person ID'].nunique()

print("NW")
#drop CMR offenses, remaining unique people
nw = nw.loc[nw['CMRoffense']=='no'].copy()
print(nw['Person ID'].nunique())

#drop people >21 in NW
nw=nw.loc[nw['Age at Offense']<21].copy()
print(nw['Person ID'].nunique())
total_people_nw = nw['Person ID'].nunique()

Suffolk
90719
NW
19686
2854


In [4]:
# In Suffolk, there is only one juvenile court is listed, and it does not appear on this list. 
# https://www.suffolkdistrictattorney.com/about-the-office/contact-directions
# "Some of these courts have juvenile sessions for offenders under the age of 18; courts without a juvenile 
# session send their cases to the Boston Juvenile Court downtown."

print("Suffolk DA courts \n", suff['Court'].value_counts())
print("\n")
print("Northwestern DA courts \n", nw['Court'].value_counts())

Suffolk DA courts 
 BMC     66433
DOR     65212
ROX     44288
CHE     35929
WROX    23391
SUP     18585
EBOS    13853
SBO     13571
BRI     11710
CHA      7853
Name: Court, dtype: int64


Northwestern DA courts 
 Belchertown District Court    3275
Greenfield District Court     1634
Northampton District Court    1288
Hadley Juvenile Court         1158
Orange District Court          799
Greenfield Juvenile Court      616
Orange Juvenile Court          401
Belchertown Juvenile Court     367
Hampshire Superior Court       178
Franklin Superior Court         87
Name: Court, dtype: int64


In [12]:
print("Suffolk - Dispositions \n")
print(suff['Disposition'].loc[suff['Expungeable']=="Yes"].value_counts(dropna=False))

print("Suffolk - % by Disposition \n")
print(suff['Disposition'].loc[suff['Expungeable']=="Yes"].value_counts(normalize=True, dropna=False)*100, "\n")

Suffolk - Dispositions 

Dismissed                   111818
Plea                         44568
NaN                          36160
Continued w/o Finding         8526
Nole Prosequi                 6169
Pre Trial Probation           3422
Verdict - Jury Trial          1814
Verdict - Bench Trial         1730
Convert to Civil               681
Filed w/o Change of Plea       276
General Continuance             91
Diversion                       74
No True Bill                    28
Bound Over                       9
Pending                          8
Name: Disposition, dtype: int64
Suffolk - % by Disposition 

Dismissed                   51.918059
Plea                        20.693306
NaN                         16.789399
Continued w/o Finding        3.958695
Nole Prosequi                2.864320
Pre Trial Probation          1.588864
Verdict - Jury Trial         0.842256
Verdict - Bench Trial        0.803254
Convert to Civil             0.316194
Filed w/o Change of Plea     0.128149
General C

In [13]:
print("NW - Dispositions \n")
print(nw['Disposition'].loc[nw['Expungeable']=="Yes"].value_counts(dropna=False))

print("NW - % by Disposition \n")
print(nw['Disposition'].loc[nw['Expungeable']=="Yes"].value_counts(normalize=True, dropna=False)*100, "\n")

NW - Dispositions 

Dismissed at Request of Comm         1442
Continued w/o Finding                1237
c276s87 finding                      1000
Nolle Prosequi                        845
Not Responsible                       762
Responsible                           512
Guilty                                330
Dismissed                             275
Dismissed on Payment                  239
Dismissed Prior to Arraignment        230
NaN                                   224
Responsible Filed                      84
Delinquent                             79
DYS Committed                          22
Guilty Filed                           21
Dismissed by Court                     18
Charge Handled as a Civil Charge       17
Accord/Satisfaction                    13
Delinquent Filed                       12
Found Incompetent                       8
Case Transferred                        5
Dismissed prior to complaint            5
CLOSED-NO CHARGES                       4
Guilty on Less